In [1]:
import glob
import pandas as pd
import numpy as np
import os, sys
import collections
import math
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import cross_val_score


from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier 

In [3]:
from numba.typed import List
from numba import jit, njit, vectorize

In [4]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [5]:
sample_rate = 50  # number of observation per second based on dataset documentation(150 samples in 3 second)

sliding_size = int((1/3) * sample_rate)  # number of skipped datapoints to start next window
print(sliding_size)

16


# Feature sets

In [6]:
@njit()
def mean_crossing_rate(col):
    # col = np.array(values)
    normalized = col - col.mean()  # to make elements of array possitive or negetive
    return ((normalized[:-1] * col[1:]) < 0).sum()  # Zero-Crossing_rate

@njit()
def iqr(window):  # inter-quartile range
    Q1 = np.median(window[:len(window)//2])  # First quartile (Q1) 
    Q3 = np.median(window[len(window)//2:])  # Third quartile (Q3) 
    IQR = Q3 - Q1 # Interquartile range (IQR) 
    return(IQR) 
@njit()
def calc_sma_for_window(data):
    return np.sum(data) / len(data)  
@njit()
def get_min(x):
    m = np.min(x)
    return m
@njit()
def get_max(x):
    m = np.max(x)
    return m
@njit()
def get_mean(x):
    m = np.mean(x)
    return m
@njit()
def get_var(x):
    m = np.var(x)
    return m
@njit()
def get_sum(x):
    m = x.sum()
    return m 
@njit()
def get_median(x):
    m = np.median(x)
    return m 
@njit()
def get_std(x):
    m = np.median(x)
    return m 
@njit()
def get_rng(x):
    n = np.max(x)
    m = np.min(x)
    z = n-m
    return z 

def get_entropy(Y):
    """
    Also known as Shanon Entropy
    Reference: https://en.wikipedia.org/wiki/Entropy_(information_theory)
    """
    unique, count = np.unique(Y, return_counts=True, axis=0)
    prob = count/len(Y)
    en = np.sum((-1)*prob*np.log2(prob))
    return en

In [7]:
def Energy(frame):
    return sum( [ abs(x)**2 for x in frame ] ) / len(frame)

# Feature extraction and Vector_Normalization on signal

In [8]:
def Features(window):# mean, std,max,min and zero-crossing-rate
    wind = window.iloc[:, :-1]
    win = np.array(wind)
    
    data_normalized = preprocessing.normalize(win, norm  = 'l1') # normalization on signal
    win = data_normalized
    features = []
    
    features.append(get_mean(win))
    features.append(get_median(win))
    features.append(get_std(win))
    features.append(get_var(win))
    features.append(get_min(win))
    features.append(get_max(win))
    features.append(get_sum(win))
    features.append(calc_sma_for_window(win))
    features.append(get_rng(win))
    features.append(get_entropy(win))
    mean_crossing = [mean_crossing_rate(win[:, i]) for i in range(win.shape[1])]
    features.append(np.array(mean_crossing))
    IQR = iqr(win)
    features.append(np.array(IQR))
    energy_measure = Energy(win)
    features.append(np.array(energy_measure))
 

    
    features = np.hstack(features).tolist()
    
    label = window.iloc[:, -1].mode()[0]  ## select the most frequent label as the label of the window
    features.append(label)
    return features

In [9]:
def windowing_dataset(dataset, win_size, feature_extraction_function, subject_id, overlap=False):
    windowed_dataset = []
    win_count = 0
    if overlap:
        step_size = sliding_size  # for Overlapping technique
    else:
        step_size = win_size  # for Non-overlapping technique

    for index in range(0, dataset.shape[0], step_size):
        start = index
        end = start + win_size
        # to assure all of windows are equal in size
        if (end <= dataset.shape[0]):
            window = dataset.iloc[start:end, :].reset_index(drop=True)
            win_count = win_count + 1
            features = feature_extraction_function(window)

            windowed_dataset.append(features)

    final = pd.DataFrame(windowed_dataset)
    final.insert(0, 'group', subject_id)  # to use in Subject CV
    return final

In [10]:
def Preprocessing(dataset_path, overlapping):
    feature_function = Features
    win_size = 3

    print("Start for win size {}".format(win_size))
    datapoints_per_window = int(win_size * sample_rate)

    print(feature_function.__name__)

    ALL = []
    List = [2,5,15]
    for subject in List:
        file_path = dataset_path + '\subject{0}_mutual5.csv'.format(subject)
        all_cols = []
        
        
#########################################################################################################################
        for i in range(2, 117, 13):# indices of accelarations
            indices = list(range(i, i + 13))
            all_cols.extend(indices)

        all_cols.append(119)  # label index

        tmp_db = pd.read_csv(file_path, header=None, usecols=all_cols, sep='\t')
        tmp_db.columns = list(range(tmp_db.shape[1]))  # re-index the columns

        transformed_db = windowing_dataset(tmp_db, datapoints_per_window, feature_function, subject,
                                                   overlap=overlapping)

        ALL.append(transformed_db)
#########################################################################################################################
       
    #final_dataset = pd.DataFrame()
    ALL_dataset = pd.DataFrame()
    ALL_dataset = ALL_dataset.append(ALL, ignore_index=True)
    
    return ALL_dataset
   

In [11]:
def subject_cross_validation(X, Y, groups, classifier):
    f1 = []
    logo = LeaveOneGroupOut()
    i = 0
    for train_index, test_index in logo.split(X, Y, groups=groups):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        i += 1
        classifier.fit(X_train, y_train)

        y_pred = classifier.predict(X_test)
        f = f1_score(y_true=y_test, y_pred=y_pred, average='micro')
        
        print('Model-',i ,' -',' f1 score: ', f)
       
        f1.append(f)
    return np.mean(f1)

In [12]:
def classifier(dataset, model):
    results = dict()
    win_size = float(3)
    print('window_size = ', win_size,' sec')

    dataset = dataset
    groups = dataset['group']
    X = dataset.iloc[:, 1:-1]
    X = np.array(X)
    
    Y = dataset.iloc[:, -1]
    Y = np.array(Y)

    for model_name, mod in model.items():
        f1 = 0

        f1 = subject_cross_validation(X, Y, groups, mod)

        if win_size in results:
            results[win_size].append(f1)
        else:
            results[win_size] = [f1]


        results = collections.OrderedDict(sorted(results.items()))

        final = []
        col = list(model.keys())
        col.insert(0, "window-size")
        final.append(col)
        for k, v in results.items():
            tmp = []
            tmp.append([k])
            tmp.append(v)
            flattened = [val for sublist in tmp for val in sublist]
            final.append(flattened)

    accuracy = final[1][1]
    
    return accuracy

In [13]:
mod = {'RF': RandomForestClassifier(n_estimators=40, random_state=42, n_jobs=-1)}

In [14]:
dataset_csv_path = r"D:\projec\proj\data_mutual5"

overlapping = 1  # input 0 for non overlapping, 1 for overlapping

ALL = Preprocessing(dataset_path=dataset_csv_path, overlapping=bool(int(overlapping)))

Start for win size 3
Features


In [15]:
ALL.head(10)

group         0         1         2         3         4         5  \
0      2 -0.001453  0.000342  0.000342  0.000304 -0.132019  0.120315   
1      2 -0.001571  0.000411  0.000411  0.000295 -0.132019  0.120315   
2      2 -0.001646  0.000492  0.000492  0.000285 -0.132019  0.120315   
3      2 -0.001688  0.000532  0.000532  0.000287 -0.148639  0.132053   
4      2 -0.001739  0.000553  0.000553  0.000284 -0.148639  0.132053   
5      2 -0.001810  0.000546  0.000546  0.000289 -0.148639  0.132053   
6      2 -0.001844  0.000518  0.000518  0.000290 -0.148639  0.132053   
7      2 -0.001761  0.000575  0.000575  0.000290 -0.158511  0.132053   
8      2 -0.001875  0.000586  0.000586  0.000293 -0.158511  0.132053   
9      2 -0.001940  0.000526  0.000526  0.000287 -0.158511  0.132053   

           6         7         8  ...       236       237       238  \
0 -25.499847 -0.169999  0.252334  ...  0.000055  0.000020  0.000016   
1 -27.566334 -0.183776  0.252334  ...  0.000113  0.000038  0.000015   
2 -28.883072 -0.192554  0.252334  ...  0.000117  0.000039  0.000015   
3 -29.622242 -0.197482  0.280692  ...  0.000127  0.000040  0.000014   
4 -30.516254 -0.203442  0.280692  ...  0.000188  0.000050  0.000014   
5 -31.759785 -0.211732  0.280692  ...  0.000192  0.000049  0.000014   
6 -32.353435 -0.215690  0.280692  ...  0.000177  0.000045  0.000015   
7 -30.907445 -0.206050  0.290564  ...  0.000189  0.000044  0.000015   
8 -32.899744 -0.219332  0.290564  ...  0.000248  0.000051  0.000014   
9 -34.048208 -0.226988  0.290564  ...  0.000237  0.000051  0.000016   

            239       240       241       242       243       244  245  
0  3.755066e-07  0.000001  0.000015  0.000002  0.000015  0.000003    0  
1  4.482091e-07  0.000001  0.000013  0.000002  0.000014  0.000003    0  
2  5.034981e-07  0.000001  0.000011  0.000003  0.000013  0.000004    0  
3  5.050558e-07  0.000001  0.000010  0.000004  0.000012  0.000004    0  
4  6.018867e-07  0.000002  0.000008  0.000005  0.000011  0.000005    0  
5  8.052065e-07  0.000002  0.000007  0.000005  0.000010  0.000006    0  
6  8.249670e-07  0.000002  0.000006  0.000006  0.000009  0.000007    0  
7  8.474858e-07  0.000002  0.000005  0.000008  0.000008  0.000007    0  
8  1.030834e-06  0.000003  0.000004  0.000009  0.000007  0.000008    0  
9  1.189469e-06  0.000002  0.000003  0.000010  0.000005  0.000009    0  

[10 rows x 247 columns]

In [16]:
print('ALL_accuracy = ', )
ALL_accuracy = classifier(dataset=ALL, model=mod)
ALL_accuracy 

ALL_accuracy = 
window_size =  3.0  sec
Model- 1  -  f1 score:  0.6624172349046349
Model- 2  -  f1 score:  0.8457542268820252
Model- 3  -  f1 score:  0.8113158408750569


0.7731624342205722

In [17]:
d ={'ALL': [ALL_accuracy]}

In [18]:
df_accuracy = pd.DataFrame(data=d)

In [19]:
df_accuracy

ALL
0  0.773162